In [9]:
from h2ogpte import H2OGPTE
import numpy as np
import pandas as pd
import os
from datetime import datetime
import re
import pathlib
import spotipy  
import requests

# Without Query 

In [10]:
# Read to get User's favourite genre
user = pd.read_csv('../../data/user.csv')
genre = user[user['UserID'] == 'sibyl'].iloc[:, -3:].to_csv(index=False, header=False).strip()
print(genre)


rock,r&b,pop


In [11]:
# In case, id fails, use backup of top ranked song 
music_genres = {
    "country": "4H2TRR9FjnnIwxGnIt9stO",
    "electronic": "6s0yNmp4Hd32wGx40T6uL8",
    "folk": "0fjYN9BylnRXMA3or3QSld",
    "hip hop": "5ByAIlEEnxYdvpnezg7HTX",
    "jazz": "7pKWTcPfT9mg2iAhobFHpS",
    "k-pop": "5FLuHcyUiAlq2wCoqVuqUa",
    "metal": "55tGzbf7CA1fnnlTy8HgDX",
    "pop": "3TGRqZ0a2l1LRblBkJoaDx",
    "r&b": "6mz1fBdKATx6qP4oP1I65G",
    "rap": "59e7E2LMPZ2bhW5G6aCwX8",
    "rock": "5QTxFnGygVM4jFQiBovmRo",
    "soul": "2sCf9tz6LHByczuVT7rqIx"
}

backup = genre.split(',')
backup = [music_genres[genre] for genre in backup if genre in music_genres]
print(backup)

['5QTxFnGygVM4jFQiBovmRo', '6mz1fBdKATx6qP4oP1I65G', '3TGRqZ0a2l1LRblBkJoaDx']


# With Query

In [12]:
client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key='sk-yG5q6JxtLgMS1Twv0mwZKbIVmJw5pNKGL6uU57p5IfYYJUq7',
)

In [13]:
# Create Collection
collection_id = client.create_collection(
    name="MusicRecco",
    description="Music Reccommender",
)

In [14]:
# Read Data
songs = pd.read_excel('/Users/blabbyduck/Desktop/LLMasters-1/data/song.xlsx')
user = pd.read_csv('/Users/blabbyduck/Desktop/LLMasters-1/data/user.csv')
history = pd.read_excel('/Users/blabbyduck/Desktop/LLMasters-1/data/listening_history.xlsx')

In [17]:

songs = pd.read_excel('../../data/song.xlsx')
user = pd.read_csv('../../data/user.csv')

In [21]:
# Upload
songs_data = client.upload('../../data/song.xlsx', open('../../data/song.xlsx', 'rb'))

In [22]:
# Ingest the uploaded data
client.ingest_uploads(collection_id, [songs_data])

Job(id='203bcc3f-edee-48e8-8f45-dd33765a44b7', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 4, 12, 6, 48, 34, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='945eced79ecf4561bfd87a573f227b13', status='Indexing done.'), JobStatus(id='e3750a471c7d4132a3c85ed52fb8282a', status='Collecting done.')], errors=[], last_update_date=datetime.datetime(2024, 4, 12, 6, 48, 56, tzinfo=TzInfo(UTC)), duration='22s', duration_seconds=22.0)

In [23]:
#Initiate Chat
chat_session_id = client.create_chat_session(collection_id)

In [50]:
# Query to get suggestion for genre
recc_query = 'I like the genres ' + genre + ',recommend me 1 song from each of the genre I like with its Spotify ID. The Spotify ID should be a 22-character alphanumeric code.'
with client.connect(chat_session_id) as session:
    answer = session.query(
        message=recc_query,
        system_prompt='Reccommend a maximum of 5 songs. Return ONLY the music name in double quotation marks.',
        rag_config={"rag_type": "rag"},
    ).content

    bot_response = answer
    
print(bot_response)

Based on the information provided, here are 3 songs that you might like, one from each of the genres you mentioned (rock, r&b, and pop), along with their corresponding Spotify IDs:

* Rock: "Born to Be Wild" by Steppenwolf (Spotify ID: 6QT53ohGZeLGoYktBPRePD)
* R&B: "I Want You" by Marvin Gaye (Spotify ID: 15cGdiAHmezm0P04jVAjMo)
* Pop: "Call Me Maybe" by Carly Rae Jepsen (Spotify ID: Carly Rae Jepsen)

Note: I had to truncate the Spotify ID for the Carly Rae Jepsen song because it was too long to fit in this response. The full ID is "4H2TRR9FjnnIwxGnIt9stO".


In [25]:
# Convert query to spotify id to get name and album picture
pattern = r'\w{22}'

spotify_ids = re.findall(pattern, bot_response)
print(spotify_ids)

# In case, id fails, use backup of top ranked song 
music_genres = {
    "country": "4H2TRR9FjnnIwxGnIt9stO",
    "electronic": "6s0yNmp4Hd32wGx40T6uL8",
    "folk": "0fjYN9BylnRXMA3or3QSld",
    "hip hop": "5ByAIlEEnxYdvpnezg7HTX",
    "jazz": "7pKWTcPfT9mg2iAhobFHpS",
    "k-pop": "5FLuHcyUiAlq2wCoqVuqUa",
    "metal": "55tGzbf7CA1fnnlTy8HgDX",
    "pop": "3TGRqZ0a2l1LRblBkJoaDx",
    "r&b": "6mz1fBdKATx6qP4oP1I65G",
    "rap": "59e7E2LMPZ2bhW5G6aCwX8",
    "rock": "5QTxFnGygVM4jFQiBovmRo",
    "soul": "2sCf9tz6LHByczuVT7rqIx"
}

backup = genre.split(',')
backup = [music_genres[genre] for genre in backup if genre in music_genres]
spotify_ids.extend(backup)
print(spotify_ids)

['3ODF56f90W3qxDqxHAUi3v', '0xZuxFQjLWZnRMOFlFDsF5']
['3ODF56f90W3qxDqxHAUi3v', '0xZuxFQjLWZnRMOFlFDsF5', '5QTxFnGygVM4jFQiBovmRo', '6mz1fBdKATx6qP4oP1I65G', '3TGRqZ0a2l1LRblBkJoaDx']


# Spotify API

In [26]:
spotify_ids = ['3ODF56f90W3qxDqxHAUi3v', '0xZuxFQjLWZnRMOFlFDsF5', '5QTxFnGygVM4jFQiBovmRo', '6mz1fBdKATx6qP4oP1I65G', '3TGRqZ0a2l1LRblBkJoaDx']

In [27]:
url = "https://accounts.spotify.com/api/token"
payload = {
    "grant_type": "client_credentials",
    "client_id": "31e6d0a4c512492685193069c8450433",
    "client_secret": "054e3779e1974c038b277f4f2b5460ae"
}
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

response = requests.post(url, data=payload, headers=headers)

access_token = response.json()['access_token']


In [28]:
def get_spotify_track_info(track_id, access_token):    
    # Spotify API endpoint for retrieving track information
    endpoint = f"https://api.spotify.com/v1/tracks/{track_id}"
    
    # Set up the request headers with the access token
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    # Make the GET request to Spotify API
    response = requests.get(endpoint, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        track_info = response.json()
        
        # Extract the track name and image URL from the response
        track_name = track_info['name']
        image_url = track_info['album']['images'][0]['url']  # Get the first image (largest size)
        
        return track_name, image_url
    else:
        # If the request failed, print the error message
        print(f"Error: {response.status_code} - {response.text}")
        return None, None


In [29]:
track_info_list = []

for track_id in spotify_ids:
    track_name, image_url = get_spotify_track_info(track_id, access_token)
    if track_name and image_url:
        track_info_list.append({"track_name": track_name, "image_url": image_url})
        if len(track_info_list) >= 3:
            break  # Stop adding more tracks once the list reaches a length of 3

print(track_info_list)

for track_info in track_info_list:
    print(f"Track Name: {track_info['track_name']}")
    print(f"Image URL: {track_info['image_url']}")
    print()  # Add a newline for better readability between track information


[{'track_name': 'Strawberry Wine', 'image_url': 'https://i.scdn.co/image/ab67616d0000b273b2145bcb28fd6dbaa1e44a83'}, {'track_name': 'Step On', 'image_url': 'https://i.scdn.co/image/ab67616d0000b2732216645273a04d24d42505a0'}, {'track_name': "(Don't Fear) The Reaper", 'image_url': 'https://i.scdn.co/image/ab67616d0000b2733ac318439ae56ce048d7bf5d'}]
Track Name: Strawberry Wine
Image URL: https://i.scdn.co/image/ab67616d0000b273b2145bcb28fd6dbaa1e44a83

Track Name: Step On
Image URL: https://i.scdn.co/image/ab67616d0000b2732216645273a04d24d42505a0

Track Name: (Don't Fear) The Reaper
Image URL: https://i.scdn.co/image/ab67616d0000b2733ac318439ae56ce048d7bf5d



# Chatbot Page

In [30]:
#Initiate Chat
chat_session_id = client.create_chat_session(collection_id)

In [31]:
with client.connect(chat_session_id) as session:
    answer = session.query(
        message= "My favourite genre is Rap, Pop and KPop. Recommend me some songs.",
        system_prompt='Reccommend songs in the songs_short.xlsx document only, and provide spotify_id after each song',
        rag_config={"rag_type": "rag"},
    ).content
    
    reccommendations = answer

# Convert to reccommend data save

In [32]:
import re

# Provided text containing song titles
text = """
Here are five songs that you might find comforting when you're feeling down, based on the information provided in the document:

1. "Yesterday" by The Beatles (page 17, row 1, spotify_id: "6FRwDxXsvSasw0y2eDArsz")
2. "Imagine" by John Lennon (page 17, row 2, spotify_id: "1bp2IO61zbQrbWNmKKxg3f")
3. "Hallelujah" by Leonard Cohen (page 17, row 3, spotify_id: "3qiyyUfYe7CRYLucrPmulD")
4. "Someone Like You" by Adele (page 18, row 1, spotify_id: "5uvosCdMlFdTXhoazkTI5R")
5. "Fix You" by Coldplay (page 18, row 2, spotify_id: "6mHOcVtsHLMuesJkswc0GZ")
"""

# Regular expression pattern to extract song titles
pattern = r'"([^"]+)"'

# Find all matches of the pattern in the text
song_titles = re.findall(pattern, text)

print(song_titles)


['Yesterday', '6FRwDxXsvSasw0y2eDArsz', 'Imagine', '1bp2IO61zbQrbWNmKKxg3f', 'Hallelujah', '3qiyyUfYe7CRYLucrPmulD', 'Someone Like You', '5uvosCdMlFdTXhoazkTI5R', 'Fix You', '6mHOcVtsHLMuesJkswc0GZ']


# Link extraction testing

In [33]:
# Youtube
def convert_to_youtube_search_query(x):
    input_string = re.sub(r'[^a-zA-Z0-9\s]', '', x)
    cleaned_string = ''.join(char for char in input_string if char.isalnum() or char.isspace())
    query = '+'.join(cleaned_string.split())
    youtube_search_query = f"https://www.youtube.com/results?search_query={query}"
    return youtube_search_query

In [34]:
print(convert_to_youtube_search_query('waifu dance !!!! love you'))

https://www.youtube.com/results?search_query=waifu+dance+love+you


In [35]:
def id_extract(text):
    lines = text.split('\n')
    lines = [line.strip() for line in lines if line.strip()]
    regex_pattern =  r'\b[A-Za-z0-9]{22}\b'

    # Extracting Spotify IDs
    spotify_ids = []
    for string in lines:
        match = re.findall(regex_pattern, string)
        if match:
            spotify_ids.append(match)
    
    #output clickable links
    for idx, spotify_id in enumerate(spotify_ids, start=1):
        link = f"https://open.spotify.com/track/{spotify_id[0]}"
        print(f"{link}")

In [36]:
def spot_extraction(msg):
    with client.connect(chat_session_id) as session:
        # Simple Question for Document Collection
            answer = session.query(
            message= msg,
            system_prompt ='Recommend not more than 5 music. MUST return spotify_id at the end of each song.',
            rag_config={
                "rag_type": "rag",
            },
            ).content
    return answer

In [37]:
ans = spot_extraction("What are some music that i can listen to when im sad")
print(ans)

Here are five songs that you might find comforting when you're feeling sad, based on the information provided in the document context:

1. "Let The Mystery Be" by 10,000 Maniacs (Folk)
   - danceability: 0.543
   - energy: 0.572
   - loudness: -13.124
   - speechiness: 0.0317
2. "Blur" by Blur (Electronic)
   - danceability: 0.585
   - energy: 0.924
   - loudness: -4.782
   - speechiness: 0.0336
3. "Moderat" by Moderat (Electronic)
   - danceability: 0.717
   - energy: 0.46
   - loudness: -11.326
   - speechiness: 0.0372
4. "Fischerspooner" by Fischerspooner (Electronic)
   - danceability: 0.607
   - energy: 0.734
   - loudness: -7.622
   - speechiness: 0.0445
5. "Hybrid" by Hybrid (Electronic)
   - danceability: 0.832
   - energy: 0.536
   - loudness: -6.736
   - speechiness: 0.0557

These songs have lower energy levels and higher danceability scores, which might help lift your mood. Additionally, they all have moderate loudness levels and low speechiness, which can create a calming a

In [38]:
ans1 = spot_extraction("What are some music that you recommend me")
print(ans1)

Based on the information provided, here are five songs I recommend:

1. "Killin' Time" by Clint Black (country) - Spotify ID: 4H2TRR9FjnnIwxGnIt9stO
2. "Check Yes Or No" by George Strait (country) - Spotify ID: 6iy4PoAuZBMvtrlDX4VxC7
3. "It's A Great Day To Be Alive" by Travis Tritt (country) - Spotify ID: 3oW6SWwGqiZSPTiAp7ZQoH
4. "Delta Dawn" by Tanya Tucker (country) - Spotify ID: 7lUE02KHkZM44BZgjCaWRO
5. "I Love A Rainy Night" by Eddie Rabbitt (country) - Spotify ID: 5EA06TsQmqmQXZiEhr1AaM

These recommendations are based on the genre and artist popularity in the given text.


In [39]:
ans2 = spot_extraction("music that i can dance to the most. Return the spotify id too")

In [40]:
print(ans2)

Based on the information provided, here are five songs that you might be able to dance to, along with their corresponding Spotify IDs:

1. "Galvanize" by The Chemical Brothers - spotify:track:4UWDRNon2wbiqDsdkfUBGi
2. "Nobody Speak (feat. Run the Jewels)" by DJ Shadow - spotify:track:4WxQGUmWBRNovxlF2g8Xdv
3. "Breathe" by The Prodigy - spotify:track:6QT53ohGZeLGoYktBPRePD
4. "In Yer Face - Bicep Remix" by Underworld - spotify:track:15cGdiAHmezm0P04jVAjMo
5. "Born Slippy (Nuxx)" by Underworld - spotify:track:5Vx3e9PipiEceifVyBzCdj

These songs are all associated with high energy and fast beats, which could make them suitable for dancing. However, personal preferences may vary, so it's always a good idea to listen to a song before deciding whether or not it's suitable for dancing.


In [41]:
def make_link(text):
    lines = text.split('\n')
    lines = [line.strip() for line in lines if line.strip()]
    regex_pattern =  r'\b[A-Za-z0-9]{22}\b'

    # Extracting Spotify IDs
    spotify_ids = []
    for string in lines:
        match = re.findall(regex_pattern, string)
        if match:
            spotify_ids.append(match)
    
    #output clickable links
    for idx, spotify_id in enumerate(spotify_ids, start=1):
        link = f"https://open.spotify.com/track/{spotify_id[0]}"
        print(f"{link}")


In [42]:
make_link(ans)

In [43]:
import re

def extract_spotify_ids(strings):
    # Regular expression pattern to match Spotify IDs
    pattern = r"(?:Spotify ID|spotify_id)[:\s]*\"?([a-zA-Z0-9_-]{22})\"?"

    # Extracting Spotify IDs
    spotify_ids = []
    for string in strings:
        match = re.search(pattern, string)
        if match:
            spotify_ids.append(match.group(1))

    return spotify_ids

# Example usage:
strings = [
    "1. \"Yesterday\" by The Beatles (page 17, row 1) - Spotify ID: 6FRwDxXsvSasw0y2eDArsz",
    "2. \"The Sound of Silence\" by Simon & Garfunkel (page 18, row 2) - Spotify ID: 5uvosCdMlFdTXhoazkTI5R",
    "3. \"Imagine\" by John Lennon (page 19, row 1) - Spotify ID: 6mHOcVtsHLMuesJkswc0GZ",
    "4. \"Hallelujah\" by Leonard Cohen (page 20, row 2) - Spotify ID: 0sgDEFq9oeC8ueOK0mR5IA",
    "5. \"Someone Like You\" by Adele (page 20, row 3) - Spotify ID: 4KfSdst7rW39C0sfhArdrz",
    "6. \"Light My Fire\" by The Doors (spotify_id: 5uvosCdMlFdTXhoazkTI5R)",
    "7. \"The Logical Song - 2010 Remastered\" by Supertramp (spotify_id: mHOcVtsHLMuesJkswc0GZ)",
    "8. \"Barracuda\" by Heart (spotify_id: 4KfSdst7rW39C0sfhArdrz)",
    "9. \"Like a Stone\" by Audioslave (spotify_id: 3YuaBvuZqcwN3CEAyyoaei)",
    "10. \"Back In Black\" by AC/DC (spotify_id: 08mG3Y1vljYA6bvDt4Wqkj)"
]

spotify_ids = extract_spotify_ids(strings)
print(spotify_ids)


['6FRwDxXsvSasw0y2eDArsz', '5uvosCdMlFdTXhoazkTI5R', '6mHOcVtsHLMuesJkswc0GZ', '0sgDEFq9oeC8ueOK0mR5IA', '4KfSdst7rW39C0sfhArdrz', '5uvosCdMlFdTXhoazkTI5R', '4KfSdst7rW39C0sfhArdrz', '3YuaBvuZqcwN3CEAyyoaei', '08mG3Y1vljYA6bvDt4Wqkj']


In [44]:
import re

# Sample longer strings
longer_strings = [
    "1. \"Yesterday\" by The Beatles (page 17, row 1) - Spotify ID: 6FRwDxXsvSasw0y2eDArsz",
    'spotify\_id: "6FRwDxXsvSasw0y2eDArsz")',
    'rt (spotify\_id: 4KfSdst7rW39C0sfhArdrz)'
]

# Regex pattern
regex_pattern = r'\b[A-Za-z0-9]{22}\b'

# Extract alphanumeric strings of length 22 from longer strings
for string in longer_strings:
    matches = re.findall(regex_pattern, string)
    for match in matches:
        print(match)


6FRwDxXsvSasw0y2eDArsz
6FRwDxXsvSasw0y2eDArsz
4KfSdst7rW39C0sfhArdrz


<>:6: SyntaxWarning: invalid escape sequence '\_'
<>:7: SyntaxWarning: invalid escape sequence '\_'
<>:6: SyntaxWarning: invalid escape sequence '\_'
<>:7: SyntaxWarning: invalid escape sequence '\_'
/var/folders/1l/pycwvg157rv_sn0s68n1_y8c0000gn/T/ipykernel_71268/4172587299.py:6: SyntaxWarning: invalid escape sequence '\_'
  'spotify\_id: "6FRwDxXsvSasw0y2eDArsz")',
/var/folders/1l/pycwvg157rv_sn0s68n1_y8c0000gn/T/ipykernel_71268/4172587299.py:7: SyntaxWarning: invalid escape sequence '\_'
  'rt (spotify\_id: 4KfSdst7rW39C0sfhArdrz)'


In [45]:
import re

def extract_spotify_id(text):
    pattern = r"(?:Spotify ID|spotify_id)[:\s]*\"?([a-zA-Z0-9_-]{22})\"?"
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None


# Test the function with your examples
strings = [
    "\"Yesterday\" by The Beatles (page 17, row 1, spotify_id: \"6FRwDxXsvSasw0y2eDArsz\")",
    "\"Imagine\" by John Lennon (page 17, row 2, spotify_id: \"1bp2IO61zbQrbWNmKKxg3f\")",
    "\"Light My Fire\" by The Doors (spotify_id: 5uvosCdMlFdTXhoazkTI5R)",
    "\"The Logical Song - 2010 Remastered\" by Supertramp (spotify_id: 6mHOcVtsHLMuesJkswc0GZ)",
    "\"Barracuda\" by Heart (spotify_id: 4KfSdst7rW39C0sfhArdrz)",
    "\"Like a Stone\" by Audioslave (spotify_id: 3YuaBvuZqcwN3CEAyyoaei)",
    "\"Back In Black\" by AC/DC (spotify_id: 08mG3Y1vljYA6bvDt4Wqkj)",
    "\"Kinetic (The Crystal Method vs. Dada Life)\" by The Crystal Method, with a danceability score of 0.616. Spotify ID: 6s0yNmp4Hd32wGx40T6uL8",
    "\"Galvanize\" by The Chemical Brothers, with a danceability score of 0.745. Spotify ID: 4bz7uB4edifWKJXSDxwHcs",
    "\"God Is a DJ\" by Faithless, with a danceability score of 0.606. Spotify ID: 1pUFYb9peWkK8m1WCKNRjp",
    "\"In Yer Face - Bicep Remix\" by 808 State, with a danceability score of 0.616. Spotify ID: 6DxKG8EHEqkWdKrFQSvm32",
    "\"Teardrop\" by Massive Attack, with a danceability score of 0.615. Spotify ID: 67Hna13dNDkZvBpTXRIaOJ",
    "\"Hallelujah\" by Leonard Cohen (page 17, row 3, spotify_id: \"3qiyyUfYe7CRYLucrPmulD\")",
    "\"Someone Like You\" by Adele (page 18, row 1, spotify_id: \"5uvosCdMlFdTXhoazkTI5R\")",
    "\"Fix You\" by Coldplay (page 18, row 2, spotify_id: \"6mHOcVtsHLMuesJkswc0GZ\")"
]

for string in strings:
    print(extract_spotify_id(string))


6FRwDxXsvSasw0y2eDArsz
1bp2IO61zbQrbWNmKKxg3f
5uvosCdMlFdTXhoazkTI5R
6mHOcVtsHLMuesJkswc0GZ
4KfSdst7rW39C0sfhArdrz
3YuaBvuZqcwN3CEAyyoaei
08mG3Y1vljYA6bvDt4Wqkj
6s0yNmp4Hd32wGx40T6uL8
4bz7uB4edifWKJXSDxwHcs
1pUFYb9peWkK8m1WCKNRjp
6DxKG8EHEqkWdKrFQSvm32
67Hna13dNDkZvBpTXRIaOJ
3qiyyUfYe7CRYLucrPmulD
5uvosCdMlFdTXhoazkTI5R
6mHOcVtsHLMuesJkswc0GZ


ignore below

In [46]:
len("6DxKG8EHEqkWdKrFQSvm32")

22

In [47]:
#convert to list of string



In [48]:
# Function to convert Spotify IDs to clickable links
def spotify_id_to_link(spotify_id):
    return f"https://open.spotify.com/track/{spotify_id}"

# Output the clickable links
for idx, spotify_id in enumerate(spotify_ids, start=1):
    link = spotify_id_to_link(spotify_id)
    print(f"{idx}. {link}")


1. https://open.spotify.com/track/6FRwDxXsvSasw0y2eDArsz
2. https://open.spotify.com/track/5uvosCdMlFdTXhoazkTI5R
3. https://open.spotify.com/track/6mHOcVtsHLMuesJkswc0GZ
4. https://open.spotify.com/track/0sgDEFq9oeC8ueOK0mR5IA
5. https://open.spotify.com/track/4KfSdst7rW39C0sfhArdrz
6. https://open.spotify.com/track/5uvosCdMlFdTXhoazkTI5R
7. https://open.spotify.com/track/4KfSdst7rW39C0sfhArdrz
8. https://open.spotify.com/track/3YuaBvuZqcwN3CEAyyoaei
9. https://open.spotify.com/track/08mG3Y1vljYA6bvDt4Wqkj
